In [11]:
import pandas as pd
import geopandas as gpd

/Users/wknq25/opt/anaconda3/envs/geocoder_py38/lib/python3.8/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/var/folders/fg/_n1n9v7x7x1bk7m6gwfkv97m0000gp/T/ipykernel_27390/1259499560.py:2: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to 

In [19]:
census = "EW_1901"

census_csv_params = {
	"sep": "\t",
	"encoding": "latin-1",
	"quoting": 3,
	"na_values": [".", " ", "-"],
	"usecols": ["RecID", "Ctry"], #add other columns from census here
	# "nrows":250
}


country = census.split("_")[0]
year = census.split("_")[1]
print(year)

if country == "EW" and year in ["1851", "1861"]:
	suffix = "_v1"
elif country == "EW" and year in ["1891", "1901", "1911"]:
	suffix = ""
else:
	suffix = ""

full_census = pd.read_csv(f"/Users/wknq25/Documents/icem/{country}{str(year)[-2:]}outputfile{suffix}.txt",**census_csv_params)

full_census

1901


,RecID,Ctry
0,1,ENG
1,2,ENG
2,3,ENG
3,4,ENG
4,5,ENG
...,...,...
32493313,32493314,SLE
32493314,32493315,SLE
32493315,32493316,SLE
32493316,32493317,SLE


In [10]:
full_census["Ctry"].value_counts()

ENG    217
WAL      6
IRL      5
SCT      5
UNK      4
RUS      3
BHS      2
DEU      1
ITA      1
SWE      1
GIB      1
SEA      1
AUS      1
USA      1
SAM      1
Name: Ctry, dtype: int64

In [58]:
geocode = pd.read_csv("analysis_alignment_test3/1901/EW/link_lkup_merged_geoms.tsv", sep = "\t", usecols=["RecID", "os_open_roads_EW_1901"])
geocode = geocode.dropna()
geocode.head()

,RecID,os_open_roads_EW_1901
7,7533748,osgb4000000023470274_100933_1120001
8,7533749,osgb4000000023470274_100933_1120001
9,7533750,osgb4000000023470274_100933_1120001
10,7533751,osgb4000000023470274_100933_1120001
11,7533752,osgb4000000023470274_100933_1120001


In [59]:
geocode_census = pd.merge(left = geocode, right = full_census, on = "RecID", how = "left")
geocode_census.head()

,RecID,os_open_roads_EW_1901,Ctry
0,7533748,osgb4000000023470274_100933_1120001,ENG
1,7533749,osgb4000000023470274_100933_1120001,ENG
2,7533750,osgb4000000023470274_100933_1120001,ENG
3,7533751,osgb4000000023470274_100933_1120001,ENG
4,7533752,osgb4000000023470274_100933_1120001,ENG


In [22]:
geocode_census["Ctry"].value_counts()

ENG    415
SCT      5
WAL      2
SEA      1
CAN      1
AUS      1
UNK      1
Name: Ctry, dtype: int64

In [60]:
import numpy as np

geocode_census["foreign"] = np.where(geocode_census["Ctry"].str.contains("ENG|WAL|SCT", na=False), "home", "foreign")

In [61]:
grouped = geocode_census.groupby(["os_open_roads_EW_1901", "Ctry"]).size()

In [62]:
grouped = geocode_census.groupby(["os_open_roads_EW_1901", "foreign"]).size()
grouped2 = (grouped / grouped.groupby(level = 0).sum() * 100).reset_index()
grouped2 = grouped2.rename(columns={0:"perc"})
grouped2

,os_open_roads_EW_1901,foreign,perc
0,osgb4000000006295135_109958_5610001,home,100.000000
1,osgb4000000006296305_110166_5660002,home,100.000000
2,osgb4000000006296306_110166_5660002,home,100.000000
3,osgb4000000006296308_110166_5660002,foreign,2.040816
4,osgb4000000006296308_110166_5660002,home,97.959184
...,...,...,...
199861,osgb5000005275853513_110977_6020003,home,95.833333
199862,osgb5000005275941557_101382_1450001,foreign,11.111111
199863,osgb5000005275941557_101382_1450001,home,88.888889
199864,osgb5000005276015361_100003_240006,foreign,11.111111


In [63]:
grouped2 = grouped2[grouped2["foreign"] != "home"].copy()
grouped2.head()

,os_open_roads_EW_1901,foreign,perc
3,osgb4000000006296308_110166_5660002,foreign,2.040816
7,osgb4000000006298026_110126_5650001,foreign,1.935484
9,osgb4000000006298027_110126_5650001,foreign,2.380952
11,osgb4000000006298028_110126_5650001,foreign,1.535836
15,osgb4000000006298673_110166_5660002,foreign,3.738318


In [64]:
geocode_census_stats = pd.merge(left = geocode_census, right = grouped2[["os_open_roads_EW_1901","perc"]], on = "os_open_roads_EW_1901", how = "left")
geocode_census_stats["perc"] = geocode_census_stats["perc"].fillna(0)
geocode_census_stats

,RecID,os_open_roads_EW_1901,Ctry,foreign,perc
0,7533748,osgb4000000023470274_100933_1120001,ENG,home,1.028278
1,7533749,osgb4000000023470274_100933_1120001,ENG,home,1.028278
2,7533750,osgb4000000023470274_100933_1120001,ENG,home,1.028278
3,7533751,osgb4000000023470274_100933_1120001,ENG,home,1.028278
4,7533752,osgb4000000023470274_100933_1120001,ENG,home,1.028278
...,...,...,...,...,...
18457338,12325747,osgb4000000025949205_104115_2680003,ENG,home,0.000000
18457339,12325748,osgb4000000025949205_104115_2680003,ENG,home,0.000000
18457340,12325749,osgb4000000025949205_104115_2680003,ENG,home,0.000000
18457341,12325750,osgb4000000025949205_104115_2680003,ENG,home,0.000000


In [65]:
geocode_census_stats = geocode_census_stats.drop_duplicates(["os_open_roads_EW_1901"])
geocode_census_stats

,RecID,os_open_roads_EW_1901,Ctry,foreign,perc
0,7533748,osgb4000000023470274_100933_1120001,ENG,home,1.028278
188,7533974,osgb4000000023487089_100933_1120001,ENG,home,0.000000
421,7534691,osgb4000000023507682_100933_1120001,ENG,home,0.000000
426,7534763,osgb4000000023707446_100933_1120001,ENG,home,0.000000
441,7534840,osgb4000000023499717_100933_1120001,ENG,home,1.834862
...,...,...,...,...,...
18457314,12325372,osgb4000000025949208_104115_2680003,ENG,home,0.000000
18457317,12325377,osgb4000000025950369_104115_2680003,ENG,home,0.000000
18457319,12325520,osgb4000000025949206_104115_2680003,WAL,home,0.000000
18457322,12325616,osgb4000000025949390_104115_2680003,ENG,home,0.000000


In [16]:
os_open_roads = gpd.read_file("output_geom_test/1901/EW/os_open_roads/os_open_roads_EW_1901.geojson")
os_open_roads.head()

,os_open_roads_EW_1901,nameTOID,name1,conparid_01-11,CEN_1901,geometry
0,osgb4000000006295134_109960_5610001,osgb4000000006295134,FOREST DRIVE,109960.0,5610001.0,MULTILINESTRING Z ((378225.000 600232.000 0.00...
1,osgb4000000006295135_109958_5610001,osgb4000000006295135,COTTONSHOPE ROAD,109958.0,5610001.0,MULTILINESTRING Z ((378191.360 603573.690 0.00...
2,osgb4000000006295144_109958_5610001,osgb4000000006295144,BELLSHIEL ROAD,109958.0,5610001.0,MULTILINESTRING Z ((381571.000 603407.000 0.00...
3,osgb4000000006295145_109958_5610001,osgb4000000006295145,BUSHMAN'S ROAD,109958.0,5610001.0,MULTILINESTRING Z ((382718.450 601490.560 0.00...
4,osgb4000000006295146_109958_5610001,osgb4000000006295146,KYLOE ROAD,109958.0,5610001.0,MULTILINESTRING Z ((382733.000 604275.000 0.00...


In [66]:
mapped = pd.merge(left = os_open_roads, right = geocode_census_stats[["os_open_roads_EW_1901", "perc"]], on = "os_open_roads_EW_1901", how = "left")
mapped.head()

,os_open_roads_EW_1901,nameTOID,name1,conparid_01-11,CEN_1901,geometry,perc
0,osgb4000000006295134_109960_5610001,osgb4000000006295134,FOREST DRIVE,109960.0,5610001.0,MULTILINESTRING Z ((378225.000 600232.000 0.00...,NaN
1,osgb4000000006295135_109958_5610001,osgb4000000006295135,COTTONSHOPE ROAD,109958.0,5610001.0,MULTILINESTRING Z ((378191.360 603573.690 0.00...,0.0
2,osgb4000000006295144_109958_5610001,osgb4000000006295144,BELLSHIEL ROAD,109958.0,5610001.0,MULTILINESTRING Z ((381571.000 603407.000 0.00...,NaN
3,osgb4000000006295145_109958_5610001,osgb4000000006295145,BUSHMAN'S ROAD,109958.0,5610001.0,MULTILINESTRING Z ((382718.450 601490.560 0.00...,NaN
4,osgb4000000006295146_109958_5610001,osgb4000000006295146,KYLOE ROAD,109958.0,5610001.0,MULTILINESTRING Z ((382733.000 604275.000 0.00...,NaN


In [68]:
mapped = mapped.dropna(subset="perc")
mapped

,os_open_roads_EW_1901,nameTOID,name1,conparid_01-11,CEN_1901,geometry,perc
1,osgb4000000006295135_109958_5610001,osgb4000000006295135,COTTONSHOPE ROAD,109958.0,5610001.0,MULTILINESTRING Z ((378191.360 603573.690 0.00...,0.000000
32,osgb4000000006296305_110166_5660002,osgb4000000006296305,CHEVIOT STREET,110166.0,5660002.0,MULTILINESTRING Z ((399180.530 627725.980 0.00...,0.000000
33,osgb4000000006296306_110166_5660002,osgb4000000006296306,RAMSEY LANE,110166.0,5660002.0,MULTILINESTRING Z ((399103.590 627978.190 0.00...,0.000000
35,osgb4000000006296308_110166_5660002,osgb4000000006296308,GLENDALE ROAD,110166.0,5660002.0,MULTILINESTRING Z ((399092.460 628135.830 0.00...,2.040816
36,osgb4000000006296309_110166_5660002,osgb4000000006296309,TENTER HILL,110166.0,5660002.0,MULTILINESTRING Z ((399213.000 628100.000 0.00...,0.000000
...,...,...,...,...,...,...,...
806479,osgb5000005275823288_109116_5240003,osgb5000005275823288,BEVERLEY ROAD,109116.0,5240003.0,"LINESTRING Z (493556.120 459067.840 0.000, 493...",0.000000
806491,osgb5000005275841203_105676_3630001,osgb5000005275841203,TALBOT STREET,105676.0,3630001.0,MULTILINESTRING Z ((388594.000 346957.000 0.00...,1.365188
806497,osgb5000005275853513_110977_6020003,osgb5000005275853513,TERAS CLIFTON / CLIFTON TERRACE,110977.0,6020003.0,"LINESTRING Z (241634.460 240596.010 0.000, 241...",4.166667
806536,osgb5000005275941557_101382_1450001,osgb5000005275941557,WAVENDON LODGE MEWS,101382.0,1450001.0,"LINESTRING Z (493067.850 238098.240 0.000, 493...",11.111111


In [ ]:
mapped.to_csv("example_mapping.tsv", sep = "\t", index = False)

In [11]:
subset = full_census[full_census["RecID"].between(22475961, 22475965)]


In [12]:
subset

,RecID,ParID,Age
22475960,22475961,10873,63.0
22475961,22475962,10873,42.0
22475962,22475963,10873,39.0
22475963,22475964,10873,38.0
22475964,22475965,10873,7.0
